In [1]:
import numpy as np 
import pandas as pd 
import re
data = pd.read_csv('edited_data.csv')
X = data['Objectives']+'. '+ data['Description']

X = X.str.replace('<ul>', '').str.replace('<li>','').str.replace('</li>','').str.replace('</ul>','').str.replace('\n','')

X = X.fillna('')

In [2]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
#nltk.download('punkt')
#nltk.download("stopwords")
data = data.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
stop_words = set(stopwords.words('english')) 
not_delete = ["but", "shan't", "wasn't", "couldn't", "didn't", "hadn't", "against", "no", "haven't", "shouldn't", "needn't", "wouldn't", "aren't", "mightn't", "won't", "isn't", "hasn't", "don't", "mustn't", "doesn't", "not"]
STOPWORDS = [w for w in stop_words if w not in not_delete]

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

X = X.apply(clean_text)
X = X.str.replace('\d+', '')
print(X)

0       contribution metu hs ankara lab protocols coll...
1       attend makerfaire hannover demonstrate first p...
2       organize biotech day present project large gro...
3       contact connect community labs europe visit ig...
4       invite technik garage german community lab aac...
                              ...                        
2148                                                     
2149     attended fourth conference china igemer commu...
2150     created week long exhibition three igem teams...
2151     host architects competition microorganism app...
2152     secure funding supplies metagenomic sequencin...
Length: 2153, dtype: object


In [3]:
porter = PorterStemmer()
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

for i in range(0,X.count()):
    X[i] = stemSentence(X[i])

for a in X:
    a = ' '.join(a.split())

In [4]:
#multilabel classifier: project tag 0.9519213258058642
from sklearn.preprocessing import MultiLabelBinarizer
from scipy import stats

y2 = data['Project Tag']
mlb = MultiLabelBinarizer()
y2 = y2.fillna("N/A")
y2 = y2.str.split(', ')

for i in range(0, len(y2)):
    for j in range(0, len(y2[i])):
        y2[i][j] = y2[i][j].replace("Community Event ", "Community Event")\
        .replace("Conference/Panel Discussion ", "Conference/Panel Discussion")\
        .replace("Educational Material ", "Educational Material")\
        .replace("Social Media ", "Social Media").replace("Survey ", "Survey")\
        .replace("Teaching Activity ", "Teaching Activity")
    
mlb_y2 = mlb.fit_transform(y2)

#print(mlb.classes_)

#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, mlb_y2, test_size=0.3, random_state=52)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.feature_extraction.text import TfidfVectorizer

'''
#Bag of Words
OneVsRest_classifier = Pipeline([
('vectorizer', CountVectorizer(ngram_range=(1, 2),strip_accents = 'unicode')),
('tfidf', TfidfTransformer(sublinear_tf=True)),
('clf', OneVsRestClassifier(SVC(C=1.0, kernel='linear')))])

OneVsRest_classifier.fit(X_train, y_train) 
predicted = OneVsRest_classifier.predict(X_test)


#exact match
print(OneVsRest_classifier.score(X_test, y_test))

#simple match
print('OneVsRestClassifier_test:'+ str(np.mean(predicted == y_test)))
'''

Tfidf_vect = TfidfVectorizer(ngram_range=(1, 2), strip_accents = 'unicode')
#X_train = Tfidf_vect.fit_transform(X_train)
#X_test = Tfidf_vect.transform(X_test)
X_tran = Tfidf_vect.fit_transform(X)

# Fit an ensemble of SVM classifier chains and take the average prediction of all the chains.
from sklearn.metrics import jaccard_score
chains2 = [ClassifierChain(SVC(C=1.0, kernel='linear'), order='random', random_state=i)
          for i in range(10)]
for chain in chains2:
    #chain.fit(X_train, y_train)
    chain.fit(X_tran, mlb_y2)
    
#Y_pred_chains = np.array([chain.predict(X_test) for chain in chains2])


#0项是mode，1项是count
#y_pred = stats.mode(Y_pred_chains)[0]


#simple match
#print('Chain_Classifier_test:'+ str(np.mean(y_pred == y_test)))

In [5]:
#OneVsRest_classifier.fit(X, mlb_y2)
new_data = pd.read_csv('bert_summary_full.csv')
X_new = new_data['original']
#X_new = new_data['summarized']
X_new = X_new.str.replace('\n', '').str.replace('\t','')
X_new = X_new.fillna('')

X_clean = X_new.apply(clean_text)
X_clean = X_clean.str.replace('\d+', '')

for i in range(X_clean.count()):
    X_clean[i] = stemSentence(X_clean[i])
    
#如果两个词中间有许多空格，删掉，换成一个空格
for X in X_clean:
    X = ' '.join(X.split())
    
X_new_tran = Tfidf_vect.transform(X_clean)

Y_pred_chains2_new = np.array([chain.predict(X_new_tran) for chain in chains2])
y_pred_new = stats.mode(Y_pred_chains2_new)[0]

#predicted1 = OneVsRest_classifier.predict(X_new)

#y_pred_new 维度不对，要取第一个元素
output = mlb.inverse_transform(y_pred_new[0])


In [6]:
asString = []
for s in output:
    st = str(s).replace('(', '').replace(')', '').replace("'", '')
    #print(len(st))
    if len(st) != 0 and st[-1] == ',':
        st = st[:-1]
    asString.append(st)
    
new_data['Project Tag Original'] = asString

#new_data['Project Tag Summarized'] = asString
#将predict得到的结果放到csv里面
new_data.to_csv("final data.csv", sep = ',')

In [7]:
y1 = data['Audience']
mlb = MultiLabelBinarizer()
y1 = y1.fillna("N/A")
y1 = y1.str.split(', ')

for i in range(0, len(y1)):
    for j in range(0, len(y1[i])):
        y1[i][j] = y1[i][j].replace('General Public ','General Public')\
        .replace('Primary School Students ','Primary School Students')\
        .replace('Primary School Students  ','Primary School Students')\
        .replace('Primary School Students   ','Primary School Students')\
        .replace('Scientific Community ', 'Scientific Community')\
        .replace('Specialized Audience ', 'Specialized Audience')\
        .replace('Specialized Audiences', 'Specialized Audience')\
        .replace('Vendors/Businesses ', 'Vendors/Businesses')\
        .replace('iGEM Teams ', 'iGEM Teams')

mlb_y1 = mlb.fit_transform(y1)
mlb.classes_


array(['General Public', 'Legislators', 'N/A', 'Primary School Students',
       'Professors', 'Scientific Community', 'Secondary School Students',
       'Specialized Audience', 'Teachers', 'University Students',
       'Vendors/Businesses', 'iGEM Teams'], dtype=object)

In [8]:
#multilabel classifier: Audience 0.9324839247439867


#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, mlb_y1, test_size=0.3, random_state=52)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.feature_extraction.text import TfidfVectorizer

# Fit an ensemble of SVM classifier chains and take the average prediction of all the chains.
from sklearn.metrics import jaccard_score
chains1 = [ClassifierChain(SVC(C=1.0, kernel='linear'), order='random', random_state=i)
          for i in range(10)]
for chain in chains1:
    #chain.fit(X_train, y_train)
    chain.fit(X_tran, mlb_y1)
    
#Y_pred_chains = np.array([chain.predict(X_test) for chain in chains])


#0项是mode，1项是count
#y_pred = stats.mode(Y_pred_chains)[0]


#simple match
#print('Chain_Classifier_test:'+ str(np.mean(y_pred == y_test)))

In [9]:
Y_pred_chains1_new = np.array([chain.predict(X_new_tran) for chain in chains1])
y_pred_new = stats.mode(Y_pred_chains1_new)[0]

#y_pred_new 维度不对，要取第一个元素
output1 = mlb.inverse_transform(y_pred_new[0])

In [10]:
asString = []
for s in output1:
    st = str(s).replace('(', '').replace(')', '').replace("'", '')
    #print(len(st))
    if len(st) != 0 and st[-1] == ',':
        st = st[:-1]
    asString.append(st)
    
new_data['Audience Original'] = asString

#new_data['Audience Summarized'] = asString
#将predict得到的结果放到csv里面
new_data.to_csv("final data.csv", sep = ',')

In [11]:
y3 = data['Goal']
mlb = MultiLabelBinarizer()
y3 = y3.fillna("N/A")
y3 = y3.str.split(', ')
for i in range(0, len(y3)):
    for j in range(0, len(y3[i])):
        y3[i][j] = y3[i][j].replace(' Educate on Synthetic Biology','Education on Synthetic Biology')\
        .replace('Educate on Synthetic Biology','Education on Synthetic Biology')\
        .replace('Education on Synthetic Biology ','Education on Synthetic Biology')\
        .replace(' Promote Awareness of Synthetic Biology','Promote Awareness of Synthetic Biology')\
        .replace('Promote Awareness of Synthetic Biology ','Promote Awareness of Synthetic Biology')\
        .replace('Discuss Ethics and Safety ','Discuss Ethics and Safety')\
        .replace('Gain Information for iGEM Project ','Gain Information for iGEM Project')\
        .replace('Educate on the iGEM program', 'Education on iGEM Program')\
        .replace('Educate on iGEM Program', 'Education on iGEM Program')\
        .replace('Gain Information for iGEM project',  'Gain Information for iGEM Project')\
        .replace('Learn Public Opinions', 'Learn Public Opinion')\
        .replace('Networking ', 'Networking')\
        .replace('Present iGEM Project ', 'Present iGEM Project')\
        .replace('Promote Awareness of Synthetic Biology ', 'Promote Awareness of Synthetic Biology')\
        .replace('Recruit iGEM Members ', 'Recruit iGEM Members')\
        .replace('Recruit iGEM members', 'Recruit iGEM Members')\
        .replace('Science Education ', 'Science Education')
mlb_y3 = mlb.fit_transform(y3)
mlb.classes_

array(['Activism for iGEM Project', 'Activism for iGEM project',
       'Discuss Ethics and Safety', 'Education on Synthetic Biology',
       'Education on iGEM Program', 'Gain Information for iGEM Project',
       'Learn Public Opinion', 'N/A', 'Networking', 'Policy Outreach',
       'Present iGEM Project', 'Promote Awareness of Synthetic Biology',
       'Recruit iGEM Members', 'Science Education',
       'Science Education(Non-SynBio Specific)'], dtype=object)

In [12]:
#multilabel classifier: Goal 0.9207430340557275

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.feature_extraction.text import TfidfVectorizer

# Fit an ensemble of SVM classifier chains and take the average prediction of all the chains.
from sklearn.metrics import jaccard_score
chains3 = [ClassifierChain(SVC(C=1.0, kernel='linear'), order='random', random_state=i)
          for i in range(10)]
for chain in chains3:
    chain.fit(X_tran, mlb_y3)

In [13]:
Y_pred_chains3_new = np.array([chain.predict(X_new_tran) for chain in chains3])
y_pred3_new = stats.mode(Y_pred_chains3_new)[0]

#y_pred_new 维度不对，要取第一个元素
output3 = mlb.inverse_transform(y_pred3_new[0])

In [14]:
asString = []
for s in output3:
    st = str(s).replace('(', '').replace(')', '').replace("'", '')
    #print(len(st))
    if len(st) != 0 and st[-1] == ',':
        st = st[:-1]
    asString.append(st)
    
new_data['Goal Original'] = asString

#new_data['Goal Summarized'] = asString
#将predict得到的结果放到csv里面
new_data.to_csv("final data.csv", sep = ',')